# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
# def calculate_mean_win_perc_entry_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     entry_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values
        
#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)

#             commission = exec_dict['COMMISSION']
#             slippage = exec_dict['SLIPPAGE'] 
#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE'] 

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)
#             signal_idxs_true = [i - 1 for i in signal_idxs]

#             buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
#             df['buy'] = 0
#             df.loc[df.index.isin(buy_idxs), 'buy'] = 1

#             sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
#             df['sell'] = 0
#             df.loc[df.index.isin(sell_idxs), 'sell'] = -1

#             df['signal'] = df['buy'] + df['sell']

#             df['new_signal'] = 0
#             df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
#             df['signal_prices'] = 0
#             df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

#             signal_list = np.zeros((2, df.shape[0]))
#             signal_list[0][1:] = df['new_signal'].values[:-1]
#             signal_list[1] = df['signal_prices'].values

#             # fixed stop and target exit testing
#             exit_list = get_exit_entry_testing1(
#                 close_prices=df['btc_close'].values, 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list,
#                 stoploss_th=50,
#                 takeprofit_th=100,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)

#             # fixed bar exit testing
#             exit_list = get_exit_entry_testing2( 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list,
#                 n_exit_bars=5
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)

#             # random exit testing
#             exit_list = get_exit_entry_testing3( 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)

#         except:

#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)

#     mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
#         append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
#     mean_win_perc_dict['Fixed_Bar_testing'].\
#         append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
#     mean_win_perc_dict['Random_Exit_testing'].\
#         append(np.mean(entry_walk_forward_dict['random_exit_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [7]:
def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
    df['signal_prices'] = 0
    df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df


## Exit testing

### Similar approach entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [9]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [10]:
# def calculate_mean_win_perc_exit_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     exit_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:

#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)

#             commission = exec_dict['COMMISSION']
#             slippage = exec_dict['SLIPPAGE'] 
#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)
#             signal_idxs_true = [i - 1 for i in signal_idxs]

#             buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
#             df['buy'] = 0
#             df.loc[df.index.isin(buy_idxs), 'buy'] = 1

#             sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
#             df['sell'] = 0
#             df.loc[df.index.isin(sell_idxs), 'sell'] = -1

#             df['signal'] = df['buy'] + df['sell']

#             df['exit_signal'] = 0
#             df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
#             df['exit_prices'] = 0
#             df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

#             exit_list = np.zeros((2, df.shape[0]))
#             exit_list[0][1:] = df['exit_signal'].values[:-1]
#             exit_list[1] = df['exit_prices'].values

#             # replacing entry with trend following entry
#             signal_list = get_entry_exit_testing1(
#                 close_prices=df['btc_close'].values,
#                 open_prices=df['btc_open'].values,
#                 n_bars=5
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['trend_entry_testing'].append(fixed_winning_percent)

#             # replacing entry with countertrend entry
#             signal_list = get_entry_exit_testing2(
#                 close_prices=df['btc_close'].values, 
#                 open_prices=df['btc_open'].values, 
#                 prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
#                 rsi_window_size=10, 
#                 rsi_threshold = 20
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['countertrend_entry_testing'].append(fixed_bar_winning_percent)

#             # random entry testing
#             signal_list = get_entry_exit_testing3(
#                 open_prices=df['btc_open'].values
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)

#         except:

#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)

#     mean_win_perc_dict['Trend_testing'].\
#         append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
#     mean_win_perc_dict['Countertrend_testing'].\
#         append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
#     mean_win_perc_dict['Random_Entry_testing'].\
#         append(np.mean(exit_walk_forward_dict['random_entry_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [11]:
def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
    df['exit_prices'] = 0
    df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df


## Core testing

### Winning percentage

In [12]:
# def calculate_mean_win_perc_core_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     core_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             core_walk_forward_dict['core_testing'].append(winning_percent)
#         except:
#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)
#     mean_win_perc_dict['Core_Testing'].\
#         append(np.mean(core_walk_forward_dict['core_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [13]:
def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df


### Performance

In [14]:
# @njit(cache=True)
# def get_drawdown(pnl_list):

#     # running_total = np.cumsum(pnl_list)
#     # max_running_total = np.maximum.accumulate(running_total)

#     running_total = np.zeros(len(pnl_list))
#     max_running_total = np.zeros(len(pnl_list))

#     for i in range(len(pnl_list)):
#         if i == 0:
#             running_total[i] = pnl_list[i]
#             max_running_total[i] = pnl_list[i]
#         else:
#             running_total[i] = running_total[i-1] + pnl_list[i]
#             max_running_total[i] = max([max_running_total[i-1], pnl_list[i]])

#     min_total = running_total[np.argmin(running_total)]

#     max_total = max_running_total[np.argmin(running_total)]

#     if max_total == 0:
#         max_total = 0.00001

#     max_drawdown = 100 * (min_total - max_total) / max_total

#     if max_drawdown == -1:
#         max_drawdown = 0

#     return max_drawdown

@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio



In [15]:
# def calculate_mean_performance(data, text_code, monkey_test=False):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     performance_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)

#             performance_walk_forward_dict['n_trades'].append(len(signal_idxs) - 1)
#             overall_pnl = np.sum(pnl_list)
#             performance_walk_forward_dict['pnl'].append(overall_pnl)
#             performance_walk_forward_dict['roi'].append(100 * overall_pnl / (trade_size * init_inv))
#             performance_walk_forward_dict['avg_drawdown'].append(exec_dict['avg_drawdown'])
#             max_dd = get_drawdown(pnl_list)
#             performance_walk_forward_dict['drawdown'].append(max_dd)
#             performance_walk_forward_dict['drawdown_dur'].append(get_drawdown_duration(pnl_list))
#             performance_walk_forward_dict['pnl_avgd_ratio'].append(exec_dict['fitness'])
#             annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
#             performance_walk_forward_dict['sharpe_ratio'].append(annualized_sharpe_ration)
#             annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
#             performance_walk_forward_dict['sortino_ratio'].append(annualized_sortino_ratio)

#             if monkey_test:
#                 pnl_mren_arr = exec_dict['pnl_mren_arr']
#                 max_dd_mren_arr = exec_dict['max_dd_mren_arr']

#                 pnl_mren_good_cases = np.sum(np.where(overall_pnl > pnl_mren_arr, 1, 0))
#                 max_dd_mren_good_cases = np.sum(np.where(max_dd < max_dd_mren_arr, 1, 0))
#                 performance_walk_forward_dict['mt_pnl'].append(100 * pnl_mren_good_cases / len(pnl_mren_arr))
#                 performance_walk_forward_dict['mt_mdd'].append(100 * max_dd_mren_good_cases / len(max_dd_mren_arr))

#         except:
#             n_not_worked += 1

#     mean_perf_dict = defaultdict(list)
#     mean_perf_dict['N_Trades'].\
#         append(np.mean(performance_walk_forward_dict['n_trades']))
#     mean_perf_dict['PNL'].\
#         append(np.mean(performance_walk_forward_dict['pnl']))
#     mean_perf_dict['ROI (%)'].\
#         append(np.mean(performance_walk_forward_dict['roi']))
#     mean_perf_dict['AVG_Drawdown'].\
#         append(np.mean(performance_walk_forward_dict['avg_drawdown']))
#     mean_perf_dict['Drawdown (%)'].\
#         append(np.mean(performance_walk_forward_dict['drawdown']))
#     mean_perf_dict['Drawdown_Duration'].\
#         append(np.mean(performance_walk_forward_dict['drawdown_dur']))
#     mean_perf_dict['PNL_AVGD_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
#     mean_perf_dict['Sharpe_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
#     mean_perf_dict['Sortino_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['sortino_ratio']))
#     mean_perf_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     if monkey_test:
#         mean_perf_dict['MT_PNL_D'].\
#             append(np.mean(performance_walk_forward_dict['mt_pnl']))
#         mean_perf_dict['MT_MDD_D'].\
#             append(np.mean(performance_walk_forward_dict['mt_mdd']))
#         # mean_perf_dict['MT_PNL_D_Min'].\
#         #     append(np.min(performance_walk_forward_dict['mt_pnl']))
#         # mean_perf_dict['MT_MDD_D_Min'].\
#         #     append(np.min(performance_walk_forward_dict['mt_mdd']))
#         # mean_perf_dict['MT_PNL_D_Max'].\
#         #     append(np.max(performance_walk_forward_dict['mt_pnl']))
#         # mean_perf_dict['MT_MDD_D_Max'].\
#         #     append(np.max(performance_walk_forward_dict['mt_mdd']))
    
#     perf_df = pd.DataFrame(mean_perf_dict)

#     return perf_df


In [16]:
def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.mean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.mean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.mean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


## Equity curve Monte Carlo simulation

In [17]:
@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array


In [18]:
def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

In [19]:
# def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

#     max_dd_list = []
#     dd_dur_list = []
#     profit_list = []
#     roi_list = []
#     binary_profit_list = []

#     for _ in range(n_runs):

#         temp_pnl_list = np.random.choice(pnl_list, size=len(pnl_list), replace=True)

#         max_dd = get_drawdown(temp_pnl_list)
#         max_dd_list.append(max_dd)

#         dd_dur = get_drawdown_duration(temp_pnl_list)
#         dd_dur_list.append(dd_dur)

#         profit_list.append(np.sum(temp_pnl_list))
#         roi_list.append(100 * np.sum(temp_pnl_list) / (init_inv * trade_size))
#         is_profit = 1 if np.sum(temp_pnl_list) > 0 else 0
#         binary_profit_list.append(is_profit)

#     mc_dict = {}
#     mc_dict['median_max_dd'] = np.median(max_dd_list)
#     mc_dict['median_dd_dur'] = np.median(dd_dur_list)
#     mc_dict['median_profit'] = np.median(profit_list)
#     mc_dict['median_return'] = np.median(roi_list)
#     mc_dict['return_dd_ratio'] = np.median(roi_list) / np.median(max_dd_list)
#     mc_dict['prob_profit'] = np.sum(binary_profit_list) / len(binary_profit_list)

#     return mc_dict

In [20]:
# def calculate_mc_performance(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     performance_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

#             performance_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
#             performance_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
#             performance_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
#             performance_walk_forward_dict['median_return'].append(mc_dict['median_return'])
#             performance_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
#             performance_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
#         except:
#             n_not_worked += 1

#     mean_perf_dict = defaultdict(list)
#     mean_perf_dict['median_drawdown (%)'].\
#         append(np.mean(performance_walk_forward_dict['median_max_dd']))
#     mean_perf_dict['median_drawdown_duration'].\
#         append(np.mean(performance_walk_forward_dict['median_dd_dur']))
#     mean_perf_dict['median_profit'].\
#         append(np.mean(performance_walk_forward_dict['median_profit']))
#     mean_perf_dict['median_ROI (%)'].\
#         append(np.mean(performance_walk_forward_dict['median_return']))
#     mean_perf_dict['ratio'].\
#         append(np.mean(performance_walk_forward_dict['return_dd_ratio']))
#     mean_perf_dict['prob'].\
#         append(np.mean(performance_walk_forward_dict['prob_profit']))
    
#     perf_df = pd.DataFrame(mean_perf_dict)

#     return perf_df


In [21]:
def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df


# Data loading function

In [49]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    # df = df.iloc[-(7 * 60 * 24 * 52):]
    df = df.iloc[(252000-2*50400): (252000-50400)]
    # df = df.iloc[(252000-50400): 252000]
    # df = df.iloc[252000: (252000+50400)]
    # df = df.iloc[(252000+50400): (252000+2*50400)]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [50]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min.csv')
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min_all.csv')
df_52w = generate_52w_data(data_path)
df_52w.head()
df_52w.iloc[0]['datetime'], df_52w.iloc[-1]['datetime']

(Timestamp('2024-02-14 18:14:00'), Timestamp('2024-03-20 18:13:00'))

## Loading strategies generated

In [24]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\derived_strategies/\ge_results_inst_ind_comb_numba.csv')
strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (2).csv")
strategy_file_path = Path(r"C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_10_30_180400_225305_2676_225305/\ge_results.csv")
strategy_file_path = 'test_strategies.csv'

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1.5)]
df_str = df_str[~df_str.duplicated()]
# df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str

,buy,sell
0,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....
1,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....
2,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...
3,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...
4,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,(price_data['btc_close'][MAX_LAG:] > get_lag(p...
...,...,...
1483,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...
1484,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] > get_lag(...
1485,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...
1486,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...


In [25]:
# buy_list = [
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_max(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_min(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] < 30)",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] > 70)",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",

# ]

# sell_list = [
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_min(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_max(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] > 70)",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] < 30)",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",
# ]

# df_str = pd.DataFrame()
# df_str['buy'] = buy_list
# df_str['sell'] = sell_list
# df_str

## Running the tests over the strategies

In [26]:
# final_entry_win_pc_df = pd.DataFrame()
# final_exit_win_pc_df = pd.DataFrame()
# final_core_win_pc_df = pd.DataFrame()
# final_perf_df = pd.DataFrame()
# final_mc_df = pd.DataFrame()

# lag_txt = '{i}'

# strategy_idx = 1

# for row in tqdm(df_str.iloc[:3000].itertuples()):#df_str.iloc[:10].itertuples()

#     buy_signal_txt = row.buy
#     sell_signal_txt = row.sell

#     txt_code = f'''import os
# CUR_DIR = os.getcwd()
# os.chdir('src')
# #import pandas as pd
# import numpy as np
# import gc
# from fitness.indicators import numba_indicators
# from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag, get_monkey_test_results
# os.chdir(CUR_DIR)
# #from numba import njit
# COMMISSION = 0.015
# SLIPPAGE = 0.00005
# AVAILABLE_CAPITAL = 700000
# TRADE_SIZE = 0.5
# MAX_LAG = 5
# buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
# if len(buy_idxs) == 0 or len(sell_idxs) == 0:
#     fitness = -999
# else:
#     buy_idxs = np.array(buy_idxs)
#     sell_idxs = np.array(sell_idxs)
#     open_prices = price_data['btc_open']
#     pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
#     buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
#     sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
#     if buy_idxs[0] < sell_idxs[0]:
#         buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
#         buy_pnl = np.sum(buy_arr)
#         sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
#         sell_pnl = np.sum(sell_arr)
#         all_arr = merge_pnl(buy_arr, sell_arr)
#     else:
#         sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
#         sell_pnl = np.sum(sell_arr)
#         buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
#         buy_pnl = np.sum(buy_arr)
#         all_arr = merge_pnl(sell_arr, buy_arr)
#     total_pnl = buy_pnl + sell_pnl
#     equity_curve_arr = np.cumsum(all_arr)
#     drawdowns = get_drawdowns(equity_curve_arr)
#     avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
#     fitness = total_pnl / avg_drawdown
#     if np.isnan(fitness) or total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
#         fitness = -999
# gc.collect()'''

#     entry_win_pc_df = calculate_mean_win_perc_entry_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})
#     entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
#     final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

#     exit_win_pc_df = calculate_mean_win_perc_exit_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
#     final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

#     core_win_pc_df = calculate_mean_win_perc_core_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
#     final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

#     perf_df = calculate_mean_performance(
#         data=df_52w, 
#         text_code=txt_code,
#         monkey_test=False
#     )

#     perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
#     final_perf_df = pd.concat([final_perf_df, perf_df])

#     # mc_df = calculate_mc_performance(
#     #     data=df_52w, 
#     #     text_code=txt_code
#     # )

#     # mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
#     # final_mc_df = pd.concat([final_mc_df, mc_df])

#     strategy_idx += 1

In [27]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

bars_per_5week = 7 * 60 * 24 * 5
n_bars_per_year = 7 * 60 * 24 * 52

if df_52w.shape[0] < n_bars_per_year:
    n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    sell_signal_txt = row.sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag, get_monkey_test_results
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
if len(buy_idxs) == 0 or len(sell_idxs) == 0:
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    if buy_idxs[0] < sell_idxs[0]:
        buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        all_arr = merge_pnl(buy_arr, sell_arr)
    else:
        sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        all_arr = merge_pnl(sell_arr, buy_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    elif total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = df_52w.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        try:
            equity_curve_arr = exec_dict['equity_curve_arr']
            equity_curve_dict[strategy_idx].append(equity_curve_arr)
        except:
            pass

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=True)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

1488it [20:26,  1.21it/s]


In [28]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True) 
final_perf_df.reset_index(drop=True, inplace=True)
final_mc_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [29]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,NaN,NaN,0.0
1,strategy2,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,NaN,NaN,0.0
2,strategy3,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,NaN,NaN,0.0
3,strategy4,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,NaN,NaN,0.0
4,strategy5,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...
1483,strategy1484,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,NaN,NaN,NaN,0.0
1484,strategy1485,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,46.608040,56.936024,50.225333,0.0
1485,strategy1486,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,49.462603,56.264232,53.423481,0.0
1486,strategy1487,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,53.292255,55.037492,43.531148,0.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [30]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
546,strategy547,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,69.904031,66.235469,64.402233,0.0
802,strategy803,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.518998,65.530607,65.536008,0.0
842,strategy843,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,69.232468,76.253213,76.670817,0.0
874,strategy875,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,65.533376,65.530607,65.525343,0.0
883,strategy884,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,100.000000,100.000000,100.000000,0.0
914,strategy915,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,65.533376,65.530607,65.529360,0.0
954,strategy955,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,65.533376,65.530607,65.531368,0.0
963,strategy964,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,76.502131,77.457519,63.436421,0.0


## Showing exit testing results

In [31]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,61.289618,60.352869,49.382888,0.0
1,strategy2,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,61.289618,60.352869,48.327115,0.0
2,strategy3,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,61.289618,60.352869,47.405600,0.0
3,strategy4,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,61.289618,60.352869,47.483827,0.0
4,strategy5,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,61.289618,60.352869,47.974998,0.0
...,...,...,...,...,...,...,...
1483,strategy1484,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.289618,60.352869,48.395674,0.0
1484,strategy1485,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,61.365035,60.376684,48.624945,0.0
1485,strategy1486,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,61.120924,60.438208,48.671390,0.0
1486,strategy1487,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,61.279695,60.462024,48.022700,0.0


### Showing strategies that have passed entry testing

In [32]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
546,strategy547,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.230079,60.497747,48.503671,0.0
802,strategy803,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.281680,60.340961,47.492707,0.0
842,strategy843,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.281680,60.352869,47.766732,0.0
874,strategy875,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.289618,60.352869,48.011826,0.0
883,strategy884,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.289618,60.352869,49.203231,0.0
914,strategy915,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.289618,60.352869,47.674626,0.0
954,strategy955,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.289618,60.352869,48.167477,0.0
963,strategy964,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,61.273741,60.352869,48.982102,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [33]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working


## Showing core testing results

In [34]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,100.0
1,strategy2,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,100.0
2,strategy3,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,100.0
3,strategy4,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,100.0
4,strategy5,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,NaN,100.0
...,...,...,...,...,...
1483,strategy1484,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,NaN,100.0
1484,strategy1485,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,82.424242,0.0
1485,strategy1486,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,38.064516,0.0
1486,strategy1487,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,87.730061,0.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [35]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
842,strategy843,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,80.0,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [36]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working


## Showing performance

In [37]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
0,strategy1,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN
1,strategy2,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN
2,strategy3,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN
3,strategy4,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN
4,strategy5,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,strategy1484,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN
1484,strategy1485,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,165.0,1.228674e+06,351.049779,2112.134066,129.318074,98.0,197.240662,798.493669,1871.859155,0.0,79.9875,51.2250
1485,strategy1486,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,155.0,2.643069e+05,75.516247,607.992095,164.647949,96.0,767.893029,242.169032,2202.430547,0.0,59.9750,47.7625
1486,strategy1487,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,163.0,1.144489e+06,326.996815,1957.332006,122.685597,96.0,251.884556,841.370930,1911.029787,0.0,79.9625,51.9625


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [38]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies)) & 
#     (final_perf_df['ROI (%)'] > 0) &
#     (final_perf_df['MT_PNL_D'] > 90) &
#     (final_perf_df['MT_MDD_D'] > 90)
# ]

In [39]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
842,strategy843,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,5.0,57590.169003,16.454334,30849.193521,182.611726,1.0,0.667279,296.529473,270.683767,0.0,62.3625,18.2625


### Showing performance for strategies that has passed all tests and have positive ROI

In [40]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D


In [41]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

In [42]:
# buy_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['buy'].tolist()

# print(buy_str_list[3])

In [43]:
# sell_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['sell'].tolist()

# print(sell_str_list[3])

### Showing Monte Carlo simulation

In [44]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,NaN,NaN,NaN,NaN,NaN,100.0
1,strategy2,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,NaN,NaN,NaN,NaN,NaN,NaN,100.0
2,strategy3,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,NaN,NaN,NaN,NaN,NaN,100.0
3,strategy4,(numba_indicators.relative_strength_index(pric...,(numba_indicators.relative_strength_index(pric...,NaN,NaN,NaN,NaN,NaN,NaN,100.0
4,strategy5,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,(price_data['btc_close'][MAX_LAG:] > get_lag(p...,NaN,NaN,NaN,NaN,NaN,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...
1483,strategy1484,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,NaN,NaN,NaN,NaN,NaN,NaN,100.0
1484,strategy1485,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,6.609599,4.0,1.227958e+06,350.845168,53.081154,1.0000,0.0
1485,strategy1486,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,23.552672,14.0,2.608422e+05,74.526336,3.164241,1.0000,0.0
1486,strategy1487,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,4.652037,3.0,1.143719e+06,326.776725,70.243786,1.0000,0.0


### Showing simulation results for strategies that has passed entry and core testing and have positive ROI

In [45]:
mc_strategies_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
842,strategy843,((numba_indicators.relative_strength_index(pri...,((numba_indicators.relative_strength_index(pri...,26.008877,1.0,57590.169003,16.454334,0.632643,0.8173,0.0


### Showing simulation results for strategies that has passed all tests and have positive ROI

In [46]:
mc_strategies_list2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working


In [47]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_btc_manual_exp100800.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_btc_manual_exp100800.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_btc_manual_exp100800.csv', index=False)
final_perf_df.to_csv('testing_results/perf_btc_manual_exp100800.csv', index=False)
final_mc_df.to_csv('testing_results/mc_btc_manual_exp100800.csv', index=False)

# Showing equity curves

In [48]:
# plt.plot(equity_curve_dict[19532][0]);